In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install ultralytics

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.3 MB/s eta 0:00:00


In [ ]:
# YOLO Model Testing on Images

import os
import random
import cv2
import numpy as np
from ultralytics import YOLO
from PIL import Image
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import glob


def setup_testing_config():
    """Configure testing parameters"""

    config = {

        'model_path': '/content/drive/MyDrive/retrained_v1/best.pt',


        'test_images_dir': '/content/drive/MyDrive/test-vehicles',

        # Inference parameters
        'confidence_threshold': 0.25,
        'iou_threshold': 0.45,
        'max_detections': 300,

        # Display settings
        'image_size': (800, 600),
        'line_thickness': 2,
        'font_size': 0.8,


        'save_results': True,          # Save annotated images
        'output_dir': '/content/drive/MyDrive/test-results-v1',
    }

    return config



def load_model(model_path):
    #Load the trained YOLO model

    print(f" Loading model from: {model_path}")

    if not os.path.exists(model_path):
        print(f" Model file not found: {model_path}")
        return None

    try:
        model = YOLO(model_path)
        print(" Model loaded successfully!")
        print(f" Model classes: {model.names}")
        return model
    except Exception as e:
        print(f" Error loading model: {str(e)}")
        return None

def get_random_test_images(test_dir, num_images=5):


    if not os.path.exists(test_dir):
        print(f" Test directory not found: {test_dir}")
        return []

    # Get all image files
    image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.JPG', '*.PNG', '*.JPEG']
    all_images = []

    for ext in image_extensions:
        all_images.extend(glob.glob(os.path.join(test_dir, ext)))

    if len(all_images) == 0:
        print(f" No images found in: {test_dir}")
        return []


    num_images = min(num_images, len(all_images))
    selected_images = random.sample(all_images, num_images)

    print(f" Found {len(all_images)} images, selected {len(selected_images)} random images")
    return selected_images

def run_inference_on_image(model, image_path, config):
    """Run inference on a single image"""

    print(f"\n🔍 Testing: {os.path.basename(image_path)}")

    #Inference
    results = model(
        image_path,
        conf=config['confidence_threshold'],
        iou=config['iou_threshold'],
        max_det=config['max_detections']
    )


    result = results[0]


    if result.boxes is not None and len(result.boxes) > 0:
        print(f" Found {len(result.boxes)} detections:")

        for i, box in enumerate(result.boxes):
            class_id = int(box.cls)
            confidence = float(box.conf)
            class_name = model.names[class_id]

            # Get bounding box coordinates
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()

            print(f"  {i+1}. {class_name}: {confidence:.3f} at [{int(x1)}, {int(y1)}, {int(x2)}, {int(y2)}]")
    else:
        print(" No detections found")

    return result

def display_results(result, image_path, config):


    # Get annotated image
    annotated_img = result.plot(
        line_width=config['line_thickness'],
        font_size=config['font_size']
    )

    # Convert BGR to RGB for display
    annotated_img_rgb = cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB)


    plt.figure(figsize=(12, 8))
    plt.imshow(annotated_img_rgb)
    plt.title(f"Detection Results: {os.path.basename(image_path)}", fontsize=14)
    plt.axis('off')
    plt.tight_layout()
    plt.show()


    if config['save_results']:
        save_annotated_image(annotated_img, image_path, config)

def save_annotated_image(annotated_img, original_path, config):

    os.makedirs(config['output_dir'], exist_ok=True)


    original_name = os.path.basename(original_path)
    name, ext = os.path.splitext(original_name)
    output_name = f"detected_{name}{ext}"
    output_path = os.path.join(config['output_dir'], output_name)


    cv2.imwrite(output_path, annotated_img)
    print(f" Saved result to: {output_path}")

def test_on_custom_image(model, image_path, config):

    if not os.path.exists(image_path):
        print(f" Image not found: {image_path}")
        return

    print(f"  Testing custom image: {image_path}")


    result = run_inference_on_image(model, image_path, config)


    display_results(result, image_path, config)

def batch_test_images(model, image_list, config):


    print(f" Running batch testing on {len(image_list)} images...")

    detection_stats = {
        'total_images': len(image_list),
        'images_with_detections': 0,
        'total_detections': 0,
        'class_counts': {}
    }

    for i, image_path in enumerate(image_list):
        print(f"\n{'='*50}")
        print(f"Image {i+1}/{len(image_list)}")


        result = run_inference_on_image(model, image_path, config)


        if result.boxes is not None and len(result.boxes) > 0:
            detection_stats['images_with_detections'] += 1
            detection_stats['total_detections'] += len(result.boxes)


            for box in result.boxes:
                class_id = int(box.cls)
                class_name = model.names[class_id]
                detection_stats['class_counts'][class_name] = detection_stats['class_counts'].get(class_name, 0) + 1


        display_results(result, image_path, config)


    print_detection_summary(detection_stats)

def print_detection_summary(stats):


    print(f"\n{'='*60}")
    print(" DETECTION SUMMARY")
    print(f"{'='*60}")
    print(f"Total images tested: {stats['total_images']}")
    print(f"Images with detections: {stats['images_with_detections']}")
    print(f"Detection rate: {stats['images_with_detections']/stats['total_images']*100:.1f}%")
    print(f"Total detections: {stats['total_detections']}")
    print(f"Average detections per image: {stats['total_detections']/stats['total_images']:.1f}")

    if stats['class_counts']:
        print(f"\n  Class Distribution:")
        for class_name, count in sorted(stats['class_counts'].items()):
            percentage = count / stats['total_detections'] * 100
            print(f"  {class_name}: {count} ({percentage:.1f}%)")



def test_random_images(num_images=5):


    print(" Testing on Random Images")
    print("="*50)


    config = setup_testing_config()


    model = load_model(config['model_path'])
    if model is None:
        return


    image_list = get_random_test_images(config['test_images_dir'], num_images)
    if not image_list:
        return


    batch_test_images(model, image_list, config)

def test_specific_image(image_path):


    print(" Testing on Specific Image")
    print("="*50)


    config = setup_testing_config()


    model = load_model(config['model_path'])
    if model is None:
        return


    test_on_custom_image(model, image_path, config)

def test_all_images_in_folder(folder_path, max_images=None):


    print(" Testing on All Images in Folder")
    print("="*50)


    config = setup_testing_config()
    config['test_images_dir'] = folder_path


    model = load_model(config['model_path'])
    if model is None:
        return


    image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.JPG', '*.PNG', '*.JPEG']
    all_images = []

    for ext in image_extensions:
        all_images.extend(glob.glob(os.path.join(folder_path, ext)))

    if max_images:
        all_images = all_images[:max_images]

    if not all_images:
        print(f" No images found in: {folder_path}")
        return


    batch_test_images(model, all_images, config)


def quick_test():


    config = setup_testing_config()
    model = load_model(config['model_path'])

    if model is None:
        return


    image_list = get_random_test_images(config['test_images_dir'], 1)
    if image_list:
        result = run_inference_on_image(model, image_list[0], config)
        display_results(result, image_list[0], config)



if __name__ == "__main__":

    print(" YOLO Traffic Detection Model Testing")
    print("="*60)

    print("\n Testing on all images in folder...")
    test_all_images_in_folder('/content/drive/MyDrive/test-vehicles', max_images=20)



In [ ]:

"""YOLO Dataset Class Remapping & Training"""

import os
import yaml
import shutil
from ultralytics import YOLO
from collections import Counter

def setup_config():


    # Original 21 classes
    ORIGINAL_CLASSES = [
        'ambulance', 'army vehicle', 'auto rickshaw', 'bicycle', 'bus',
        'car', 'garbagevan', 'human hauler', 'minibus', 'minivan',
        'motorbike', 'pickup', 'policecar', 'rickshaw', 'scooter',
        'suv', 'taxi', 'three wheelers -CNG-', 'truck', 'van', 'wheelbarrow'
    ]

    # New 5 grouped classes
    NEW_CLASSES = [
        'two-wheeler',        # ID: 0
        'car',               # ID: 1
        'auto-rickshaw',     # ID: 2
        'heavy-vehicle',     # ID: 3
        'emergency-vehicle'  # ID: 4
    ]

    # Mapping from original classes to new grouped classes
    CLASS_MAPPING = {

        'bicycle': 'two-wheeler',
        'motorbike': 'two-wheeler',
        'scooter': 'two-wheeler',
        'rickshaw': 'two-wheeler',
        'human hauler': 'two-wheeler',
        'wheelbarrow': 'two-wheeler',


        'car': 'car',
        'taxi': 'car',
        'suv': 'car',
        'pickup': 'car',
        'minivan': 'car',
        'van': 'car',


        'auto rickshaw': 'auto-rickshaw',
        'three wheelers -CNG-': 'auto-rickshaw',



        'ambulance': 'emergency-vehicle',
        'army vehicle': 'emergency-vehicle',
        'policecar': 'emergency-vehicle',


        'bus': 'heavy-vehicle',
        'garbagevan': 'heavy-vehicle',
        'minibus': 'heavy-vehicle',
        'truck': 'heavy-vehicle',
    }

    config = {

        'original_dataset_path': '/content/drive/MyDrive/indian-vehicles-with-ambulance',
        'remapped_dataset_path': '/content/drive/MyDrive/indian-vehicles-with-ambulance-remapped',


        'original_classes': ORIGINAL_CLASSES,
        'new_classes': NEW_CLASSES,
        'class_mapping': CLASS_MAPPING,

        # Training parameters
        'model_size': 'yolov8s.pt',
        'epochs': 150,
        'batch_size': 16,
        'image_size': 640,
        'learning_rate': 0.01,
        'patience': 30,
        'workers': 8,


        'project_name': 'traffic_detection_5class',
        'run_name': 'grouped_classes_v1',
    }

    return config



class DatasetRemapper:
    def __init__(self, config):
        self.config = config
        self.class_mapping = config['class_mapping']
        self.original_classes = config['original_classes']
        self.new_classes = config['new_classes']


        self.original_name_to_id = {name: idx for idx, name in enumerate(self.original_classes)}
        self.new_name_to_id = {name: idx for idx, name in enumerate(self.new_classes)}


        self.id_mapping = {}
        for orig_name, new_name in self.class_mapping.items():
            if orig_name in self.original_name_to_id:
                orig_id = self.original_name_to_id[orig_name]
                new_id = self.new_name_to_id[new_name]
                self.id_mapping[orig_id] = new_id

        print(f" Class Mapping Summary:")
        print(f"   Original: {len(self.original_classes)} classes")
        print(f"   New: {len(self.new_classes)} classes")
        print(f"   Mapped: {len(self.id_mapping)} class mappings")

    def remap_dataset(self):


        print("\n Starting dataset remapping...")

        original_path = self.config['original_dataset_path']
        remapped_path = self.config['remapped_dataset_path']


        os.makedirs(remapped_path, exist_ok=True)


        total_labels = 0
        remapped_labels = 0
        class_counts = Counter()


        for split in ['train', 'val', 'test']:
            print(f"\n Processing {split} split...")


            orig_images_dir = os.path.join(original_path, split, 'images')
            orig_labels_dir = os.path.join(original_path, split, 'labels')

            new_images_dir = os.path.join(remapped_path, split, 'images')
            new_labels_dir = os.path.join(remapped_path, split, 'labels')


            os.makedirs(new_images_dir, exist_ok=True)
            os.makedirs(new_labels_dir, exist_ok=True)

            if not os.path.exists(orig_images_dir) or not os.path.exists(orig_labels_dir):
                print(f"    Skipping {split} - directories don't exist")
                continue


            image_files = [f for f in os.listdir(orig_images_dir)
                          if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]

            split_total = 0
            split_remapped = 0

            for img_file in image_files:

                shutil.copy2(
                    os.path.join(orig_images_dir, img_file),
                    os.path.join(new_images_dir, img_file)
                )


                label_file = os.path.splitext(img_file)[0] + '.txt'
                orig_label_path = os.path.join(orig_labels_dir, label_file)
                new_label_path = os.path.join(new_labels_dir, label_file)

                if os.path.exists(orig_label_path):
                    labels_remapped = self.remap_label_file(orig_label_path, new_label_path)
                    split_total += labels_remapped['total']
                    split_remapped += labels_remapped['remapped']


                    for class_name, count in labels_remapped['class_counts'].items():
                        class_counts[class_name] += count

            total_labels += split_total
            remapped_labels += split_remapped

            print(f"   {split}: {len(image_files)} images, {split_remapped}/{split_total} labels remapped")


        self.create_remapped_yaml(remapped_path)


        print(f"\n Remapping Summary:")
        print(f"   Total labels processed: {total_labels}")
        print(f"   Labels successfully remapped: {remapped_labels}")
        print(f"   Remapping success rate: {remapped_labels/total_labels*100:.1f}%")

        print(f"\n  New class distribution:")
        for class_name in self.new_classes:
            count = class_counts.get(class_name, 0)
            print(f"   {class_name}: {count} instances")

        return remapped_path

    def remap_label_file(self, orig_file, new_file):


        stats = {
            'total': 0,
            'remapped': 0,
            'class_counts': Counter()
        }

        try:
            with open(orig_file, 'r') as f:
                lines = f.readlines()

            remapped_lines = []

            for line in lines:
                line = line.strip()
                if not line:
                    continue

                parts = line.split()
                if len(parts) >= 5:
                    stats['total'] += 1

                    # Get original class ID and map to new ID
                    orig_class_id = int(parts[0])

                    if orig_class_id in self.id_mapping:
                        new_class_id = self.id_mapping[orig_class_id]
                        new_class_name = self.new_classes[new_class_id]

                        # Create new line with remapped class ID
                        new_line = f"{new_class_id} {' '.join(parts[1:])}"
                        remapped_lines.append(new_line)

                        stats['remapped'] += 1
                        stats['class_counts'][new_class_name] += 1
                    else:
                        print(f"    Warning: Class ID {orig_class_id} not found in mapping")

            # Write remapped labels
            with open(new_file, 'w') as f:
                for line in remapped_lines:
                    f.write(line + '\n')

        except Exception as e:
            print(f"   Error processing {orig_file}: {e}")

        return stats

    def create_remapped_yaml(self, dataset_path):


        yaml_content = {
            'path': os.path.abspath(dataset_path),
            'train': 'train/images',
            'val': 'val/images',
            'test': 'test/images',
            'nc': len(self.new_classes),
            'names': self.new_classes
        }

        yaml_path = os.path.join(dataset_path, 'data.yaml')

        with open(yaml_path, 'w') as f:
            yaml.dump(yaml_content, f, default_flow_style=False)

        print(f"\n Dataset YAML created: {yaml_path}")


        print("\n YAML Content:")
        print(f"   Classes ({len(self.new_classes)}): {self.new_classes}")
        print(f"   Path: {yaml_content['path']}")

        return yaml_path


def train_from_scratch(config, dataset_yaml):

    print(f"\n Training YOLO from scratch...")
    print(f" Model: {config['model_size']}")
    print(f"  Classes: {len(config['new_classes'])}")
    print(f" Dataset: {dataset_yaml}")

    # Initialize model
    model = YOLO(config['model_size'])


    results = model.train(
        data=dataset_yaml,
        epochs=config['epochs'],
        batch=config['batch_size'],
        imgsz=config['image_size'],
        lr0=config['learning_rate'],
        patience=config['patience'],
        workers=config['workers'],


        project=config['project_name'],
        name=config['run_name'],


        save_period=10,
        plots=True,
        val=True,
        save=True,
        device='cuda' if os.path.exists('/dev/nvidia0') else 'cpu',


        optimizer='AdamW',
        close_mosaic=10,
        amp=True,
    )

    print("\n Training completed!")
    print(f" Best mAP50: {results.results_dict.get('metrics/mAP50(B)', 'N/A')}")
    print(f" Best mAP50-95: {results.results_dict.get('metrics/mAP50-95(B)', 'N/A')}")
    print(f" Model saved to: {results.save_dir}")

    return model, results


def main():


    print(" YOLO Traffic Detection - 5 Class Training")
    print("=" * 60)


    config = setup_config()

    print(f" Remapping Strategy:")
    print(f"   21 classes → 5 grouped classes")


    print(f"\n  Class Groupings:")
    for new_class in config['new_classes']:
        orig_classes = [k for k, v in config['class_mapping'].items() if v == new_class]
        print(f"   {new_class}: {orig_classes}")


    remapper = DatasetRemapper(config)
    remapped_dataset_path = remapper.remap_dataset()


    dataset_yaml = os.path.join(remapped_dataset_path, 'data.yaml')


    model, results = train_from_scratch(config, dataset_yaml)


def quick_setup():

    config = setup_config()


    config.update({
        'original_dataset_path': '/content/drive/MyDrive/indian-vehicles-with-ambulance',
        'remapped_dataset_path': '/content/drive/MyDrive/indian-vehicles-with-ambulance-remapped',
        'project_name': '/content/drive/MyDrive/training-results',
    })

    return config


if __name__ == "__main__":

    main()


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
🚗 YOLO Traffic Detection - 5 Class Training
🎯 Remapping Strategy:
   21 classes → 5 grouped classes
   Emergency vehicles get highest priority!

🏷️  Class Groupings:
   two-wheeler: ['bicycle', 'motorbike', 'scooter', 'rickshaw', 'human hauler', 'wheelbarrow']
   car: ['car', 'taxi', 'suv', 'pickup', 'minivan', 'van']
   auto-rickshaw: ['auto rickshaw', 'three wheelers -CNG-']
   heavy-vehicle: ['bus', 'garbagevan', 'minibus', 'truck']
   emergency-vehicle: ['ambulance', 'army vehicle', 'policecar']
📊 Class Mapping Summary:
   Original: 21 classes
   New: 5 classes
   Mapped: 21 class mappings

🔄 Starting dataset remapping...

📁 Processing train split...
   ✅ train: 1891 images, 1

100%|██████████| 21.5M/21.5M [00:00<00:00, 283MB/s]


Ultralytics 8.3.170 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/indian-vehicles-with-ambulance-remapped/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=grouped_classes_v1, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience

100%|██████████| 755k/755k [00:00<00:00, 21.7MB/s]

Overriding model.yaml nc=80 with nc=5

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

...

    150/150      4.23G     0.8446      0.483     0.9247        104        640: 100%|██████████| 119/119 [00:38<00:00,  3.11it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:06<00:00,  2.80it/s]

                   all        540       4189      0.736      0.561      0.633      0.399



150 epochs completed in 1.912 hours.
Optimizer stripped from traffic_detection_5class/grouped_classes_v1/weights/last.pt, 22.5MB
Optimizer stripped from traffic_detection_5class/grouped_classes_v1/weights/best.pt, 22.5MB

Validating traffic_detection_5class/grouped_classes_v1/weights/best.pt...
Ultralytics 8.3.170 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 11,127,519 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:07<00:00,  2.42it/s]


                   all        540       4189      0.772      0.595      0.654      0.406
           two-wheeler        308       1167      0.743      0.565      0.619      0.339
                   car        370       1494      0.813      0.673      0.748      0.482
         auto-rickshaw        226        573      0.834      0.609       0.72      0.443
         heavy-vehicle        362        926      0.789      0.644      0.732      0.492
     emergency-vehicle         22         29      0.682      0.483      0.452      0.271
Speed: 0.2ms preprocess, 3.1ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to traffic_detection_5class/grouped_classes_v1

✅ Training completed!
📈 Best mAP50: 0.6544711422763252
📈 Best mAP50-95: 0.40550684401844983
💾 Model saved to: traffic_detection_5class/grouped_classes_v1

🎉 All done! Your 5-class traffic detection model is ready!

🚦 Perfect for traffic signal priority systems:
   🚨 emergency-vehicle → Immediate green light
   🚛 heavy-ve